In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

root = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(0, os.path.join(root, 'src'))

from models import models_dir

In [18]:
alpha_list = os.listdir(f'{models_dir}TSOC')
alpha_list = [float(alpha.split('=')[1]) for alpha in alpha_list]

In [20]:
epochs = 1000
trained_models = {}
for alpha in alpha_list:
    trained_models_alpha = os.listdir(os.path.join(f'{models_dir}TSOC', f'alpha={alpha}'))
    trained_models[alpha] = [trained_model for trained_model in trained_models_alpha if f'epochs={epochs}' in trained_model]

In [21]:
def parse_parameters(model_name):
    model_name = model_name.split('.pth')[0]
    model_name = model_name.split('TSOC-')[1]
    params = model_name.split('_')
    param_dict = {}
    for param in params:
        key, value = param.split('=')
        # Attempt to convert to int, float, or keep as string
        if value.isdigit():
            param_dict[key] = int(value)
        else:
            try:
                param_dict[key] = float(value)
            except ValueError:
                # Special case for booleans
                if value.lower() in ['true', 'false']:
                    param_dict[key] = value.lower() == 'true'
                else:
                    param_dict[key] = value

    return param_dict

## TSOC reconstruction evaluation

### Evaluate models

In [14]:
for alpha in alpha_list:
    print(f'alpha={alpha}\n')
    for model_name in trained_models[alpha]:
        params = parse_parameters(model_name)
        orthogonal_flag = "--orthogonal" if params['ort'] else ""
        heart_rate = params['hr'].replace('-', ' ')
        command = f"""
            python ../experiments/decoder_evaluation.py \
                --n {params['n']} \
                --m {params['m']} \
                --seed_training {params['seedtrain']} \
                --mode {params['mode']} \
                --seed_matrix {params['seedmat']} \
                --alpha {alpha} \
                --isnr {params['isnr']} \
                --N_train {params['N']} \
                --train_fraction {params['tf']} \
                --N_test 10000 \
                --basis sym6 \
                --fs {params['fs']} \
                --heart_rate {heart_rate} \
                {orthogonal_flag} \
                --source {params['src']} \
                --processes 48 \
                --gpu 3 \
                --min_lr {params['minlr']} \
                --min_delta {params['mind']} \
                --patience {params['p']} \
                --batch_size {params['bs']} \
                --threshold {params['th']} \
                --epochs {params['epochs']} \
                --lr {params['lr']} \
                --optimizer {params['opt']}
            """
        os.system(command)

INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=128, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=rakeness, orthogonal=True, source=best, seed_matrix=88219, alpha=0.1, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=1e-05, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


P: 35.038  TP: 26.799  TPR: 0.77  TNR: 0.999  ACC: 0.935


100%|██████████| 10000/10000 [00:03<00:00, 2668.08it/s]
INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=128, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=rakeness, orthogonal=True, source=best, seed_matrix=88219, alpha=0.05, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=1e-05, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


P: 35.038  TP: 25.953  TPR: 0.746  TNR: 1.0  ACC: 0.929


100%|██████████| 10000/10000 [00:03<00:00, 2956.90it/s]
INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=128, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=rakeness, orthogonal=True, source=best, seed_matrix=88219, alpha=0.01, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=1e-05, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


...

Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25
has already been evaluated


INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=64, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=standard, orthogonal=False, source=random, seed_matrix=0, alpha=0.5, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=random_ort=False_seedmat=0_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0
has already been evaluated


INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=64, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=standard, orthogonal=True, source=best, seed_matrix=52744, alpha=0.5, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0
has already been evaluated


### Aggregate performance results

In [22]:
for alpha in alpha_list:
    results_folder = '/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction'
    print(f'alpha={alpha}\n')
    print(os.listdir(os.path.join(results_folder, f'alpha={alpha}')))

alpha=0.1

['TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=88219_epochs=1000_bs=128_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=1e-05_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25.pkl']
alpha=0.05

['TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=88219_epochs=1000_bs=128_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=1e-05_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25.pkl']
alpha=0.25



FileNotFoundError: [Errno 2] No such file or directory: '/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction/alpha=0.25'

In [24]:
m_list = [32, 48, 64]
mode_list = ['std', 'rak']
ort_list = [True, False]
seed_mat_list = []
seed_train_list = [0, 1, 2, 3, 4]
opt_list = ['adam', 'sgd']
bs_list = [32, 64, 128]
min_delta_list = [1e-4, 1e-5]

index = pd.MultiIndex.from_product(
    [m_list, mode_list, ort_list, seed_mat_list, seed_train_list, opt_list, bs_list, alpha_list, min_delta_list],
      names=['m', 'mode', 'ort', 'seedmat', 'seedtrain', 'opt', 'bs', 'alpha', 'mind'])

In [25]:
results = pd.DataFrame(index=index, columns=['P', 'TP', 'TPR', 'TNR', 'ACC', 'RSNR'], dtype=np.float64)
# results = pd.DataFrame(columns=['P', 'TP', 'TPR', 'TNR', 'ACC', 'RSNR'], dtype=np.float64)
for alpha in alpha_list:
    for model_name in trained_models[alpha]:
        params = parse_parameters(model_name)
        results_path = os.path.join(results_folder, f'alpha={alpha}', f"{model_name.split('.pth')[0]}.pkl")
        if os.path.exists(results_path):
            result = pd.read_pickle(results_path)
            results.loc[params['m'], params['mode'], params['ort'], params['seedmat'], params['seedtrain'], params['opt'], params['bs'], alpha, params['mind']] = result

In [26]:
# min delta
results.xs(key=30808, level='seedmat').xs(key=64, level='bs').sort_values('RSNR', ascending=False)

P         TP       TPR  \
m  mode     ort  seedtrain opt  alpha mind                                      
48 rakeness True 1         adam 0.5   0.00001  35.219447  29.312898  0.835748   
                 0         adam 0.5   0.00001  35.219447  29.344447  0.836768   
                 1         adam 0.5   0.00010  35.219447  29.294785  0.835287   
                 2         adam 0.5   0.00001  35.219447  29.311604  0.835791   
                 0         adam 0.5   0.00010  35.219447  29.273587  0.834667   

                                                    TNR       ACC       RSNR  
m  mode     ort  seedtrain opt  alpha mind                                    
48 rakeness True 1         adam 0.5   0.00001  0.987916  0.944900  32.546757  
                 0         adam 0.5   0.00001  0.987927  0.945150  32.546387  
                 1         adam 0.5   0.00010  0.987997  0.944823  32.532460  
                 2         adam 0.5   0.00001  0.987846  0.944837  32.528674  
                 0         adam 0.5   0.00010  0.987873  0.944559  32.521255

In [31]:
# batch size
results.xs(key=30808, level='seedmat').xs(key=1e-5, level='mind').sort_values('RSNR', ascending=False)

P         TP       TPR  \
m  mode     ort  seedtrain opt  bs  alpha                                   
48 rakeness True 0         adam 128 0.5    35.222409  29.461828  0.839948   
                 1         adam 128 0.5    35.222409  29.463608  0.839995   
                 2         adam 128 0.5    35.222409  29.439280  0.839338   
                 3         adam 128 0.5    35.222409  29.433347  0.839146   
                 1         adam 64  0.5    35.219447  29.312898  0.835748   
                 0         adam 64  0.5    35.219447  29.344447  0.836768   
                 2         adam 64  0.5    35.219447  29.311604  0.835791   
                 0         adam 32  0.5    35.217951  29.196985  0.832486   

                                                TNR       ACC       RSNR  
m  mode     ort  seedtrain opt  bs  alpha                                 
48 rakeness True 0         adam 128 0.5    0.988268  0.946287  32.628118  
                 1         adam 128 0.5    0.988369  0.946376  32.621216  
                 2         adam 128 0.5    0.988312  0.946143  32.614950  
                 3         adam 128 0.5    0.988239  0.946041  32.613052  
                 1         adam 64  0.5    0.987916  0.944900  32.546757  
                 0         adam 64  0.5    0.987927  0.945150  32.546387  
                 2         adam 64  0.5    0.987846  0.944837  32.528674  
                 0         adam 32  0.5    0.987760  0.943901  32.473391

In [33]:
results.xs(key=88219, level='seedmat').xs(key=1e-5, level='mind').xs(key=128, level='bs').sort_values('RSNR', ascending=False)

P         TP       TPR  \
m  mode     ort  seedtrain opt  alpha                                  
48 rakeness True 0         adam 0.50   35.03837  29.324763  0.840735   
                 2         adam 0.50   35.03837  29.333762  0.840930   
                 1         adam 0.50   35.03837  29.318335  0.840520   
                 0         adam 0.10   35.03837  26.798556  0.769953   
                                0.05   35.03837  25.953422  0.745928   
                                0.01   35.03837  24.412777  0.701778   

                                            TNR       ACC       RSNR  
m  mode     ort  seedtrain opt  alpha                                 
48 rakeness True 0         adam 0.50   0.987978  0.946424  32.530562  
                 2         adam 0.50   0.988008  0.946518  32.522389  
                 1         adam 0.50   0.987808  0.946247  32.521453  
                 0         adam 0.10   0.998793  0.934715  32.014234  
                                0.05   0.999522  0.928661  31.712007  
                                0.01   0.999891  0.916905  30.937529

In [10]:
results.xs(key=128, level='bs').sort_values('RSNR', ascending=False)

P         TP  \
m  mode     ort  seedmat seedtrain opt  mind                            
48 rakeness True 30808   0         adam 0.00001  35.222409  29.461828   
                         1         adam 0.00001  35.222409  29.463608   
                         2         adam 0.00001  35.222409  29.439280   
                         3         adam 0.00001  35.222409  29.433347   
                 77732   1         adam 0.00001  35.017900  29.305281   
                 88219   0         adam 0.00001  35.038370  29.324763   
                 75680   3         adam 0.00001  35.109771  29.313983   
                 88219   2         adam 0.00001  35.038370  29.333762   
                         1         adam 0.00001  35.038370  29.318335   
                 75680   0         adam 0.00001  35.109771  29.315862   
                         2         adam 0.00001  35.109771  29.314577   
                 77732   0         adam 0.00001  35.017900  29.245945   
                         2         adam 0.00001  35.017900  29.234177   
                 75680   1         adam 0.00001  35.109771  29.294205   

                                                      TPR       TNR       ACC  \
m  mode     ort  seedmat seedtrain opt  mind                                    
48 rakeness True 30808   0         adam 0.00001  0.839948  0.988268  0.946287   
                         1         adam 0.00001  0.839995  0.988369  0.946376   
                         2         adam 0.00001  0.839338  0.988312  0.946143   
                         3         adam 0.00001  0.839146  0.988239  0.946041   
                 77732   1         adam 0.00001  0.840939  0.988395  0.946731   
                 88219   0         adam 0.00001  0.840735  0.987978  0.946424   
                 75680   3         adam 0.00001  0.838726  0.988454  0.946140   
                 88219   2         adam 0.00001  0.840930  0.988008  0.946518   
                         1         adam 0.00001  0.840520  0.987808  0.946247   
                 75680   0         adam 0.00001  0.838741  0.988267  0.946019   
                         2         adam 0.00001  0.838725  0.988322  0.946048   
                 77732   0         adam 0.00001  0.839264  0.988349  0.946241   
                         2         adam 0.00001  0.838933  0.988323  0.946139   
                 75680   1         adam 0.00001  0.838148  0.988275  0.945859   

                                                      RSNR  
m  mode     ort  seedmat seedtrain opt  mind                
48 rakeness True 30808   0         adam 0.00001  32.628118  
                         1         adam 0.00001  32.621216  
                         2         adam 0.00001  32.614950  
                         3         adam 0.00001  32.613052  
                 77732   1         adam 0.00001  32.542928  
                 88219   0         adam 0.00001  32.530562  
                 75680   3         adam 0.00001  32.524706  
                 88219   2         adam 0.00001  32.522389  
                         1         adam 0.00001  32.521453  
                 75680   0         adam 0.00001  32.521272  
                         2         adam 0.00001  32.514645  
                 77732   0         adam 0.00001  32.510246  
                         2         adam 0.00001  32.505826  
                 75680   1         adam 0.00001  32.502708

In [28]:

results.loc[48].sort_values('RSNR', ascending=False)

P         TP  \
mode     ort   seedmat seedtrain opt  bs  alpha mind                            
rakeness True  30808   0         adam 128 0.50  0.00001  35.222409  29.461828   
                       1         adam 128 0.50  0.00001  35.222409  29.463608   
                       2         adam 128 0.50  0.00001  35.222409  29.439280   
                       3         adam 128 0.50  0.00001  35.222409  29.433347   
                       1         adam 64  0.50  0.00001  35.219447  29.312898   
                       0         adam 64  0.50  0.00001  35.219447  29.344447   
               77732   1         adam 128 0.50  0.00001  35.017900  29.305281   
               30808   1         adam 64  0.50  0.00010  35.219447  29.294785   
               88219   0         adam 128 0.50  0.00001  35.038370  29.324763   
               30808   2         adam 64  0.50  0.00001  35.219447  29.311604   
               75680   3         adam 128 0.50  0.00001  35.109771  29.313983   
               88219   2         adam 128 0.50  0.00001  35.038370  29.333762   
                       1         adam 128 0.50  0.00001  35.038370  29.318335   
               75680   0         adam 128 0.50  0.00001  35.109771  29.315862   
               30808   0         adam 64  0.50  0.00010  35.219447  29.273587   
               75680   2         adam 128 0.50  0.00001  35.109771  29.314577   
               77732   0         adam 128 0.50  0.00001  35.017900  29.245945   
                       2         adam 128 0.50  0.00001  35.017900  29.234177   
               75680   1         adam 128 0.50  0.00001  35.109771  29.294205   
               30808   0         adam 32  0.50  0.00001  35.217951  29.196985   
               75680   2         adam 64  0.50  0.00001  35.113654  29.238754   
               88219   0         adam 64  0.50  0.00001  35.039809  29.206011   
               77732   0         adam 64  0.50  0.00001  35.013635  29.155454   
               88219   1         adam 64  0.50  0.00001  35.039809  29.218352   
               77732   1         adam 64  0.50  0.00001  35.013635  29.167396   
               75680   1         adam 64  0.50  0.00001  35.113654  29.215267   
                       0         adam 64  0.50  0.00001  35.113654  29.190983   
               30808   0         sgd  50  0.50  0.00010  35.217199  29.159999   
               77732   0         adam 32  0.50  0.00001  35.011482  29.069089   
               75680   0         adam 32  0.50  0.00001  35.115615  29.102037   
               88219   0         adam 32  0.50  0.00001  35.040535  29.092851   
                                      128 0.10  0.00001  35.038370  26.798556   
                                          0.05  0.00001  35.038370  25.953422   
standard True  52744   0         adam 64  0.50  0.00010  34.463575  28.784733   
                                 sgd  50  0.50  0.00010  34.466099  28.725199   
rakeness True  88219   0         adam 128 0.01  0.00001  35.038370  24.412777   
standard False 0       0         sgd  50  0.50  0.00010  34.746199  28.941499   
                                 adam 64  0.50  0.00010  34.750995  28.851015   

                                                              TPR       TNR  \
mode     ort   seedmat seedtrain opt  bs  alpha mind                          
rakeness True  30808   0         adam 128 0.50  0.00001  0.839948  0.988268   
                       1         adam 128 0.50  0.00001  0.839995  0.988369   
                       2         adam 128 0.50  0.00001  0.839338  0.988312   
                       3         adam 128 0.50  0.00001  0.839146  0.988239   
                       1         adam 64  0.50  0.00001  0.835748  0.987916   
                       0         adam 64  0.50  0.00001  0.836768  0.987927   
               77732   1         adam 128 0.50  0.00001  0.840939  0.988395   
               30808   1         adam 64  0.50  0.00010  0.835287  0.987997   
               88219   0         a

In [85]:
import torch
print(torch.__version__)

2.5.1
